In [1]:
import keras

from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150,150,3))

In [2]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = "C:/Users/yang/Desktop/nongzuowu/small/"

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir,'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1)*batch_size] = features_batch
        labels[i*batch_size:(i+1)*batch_size] = labels_batch
        i+=1
        if i * batch_size>=sample_count:
            break
    return features, labels

In [3]:
train_features, train_labels = extract_features(train_dir, 2540)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)


Found 2540 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
train_features = np.reshape(train_features, (2540, 4*4*512))
validation_features = np.reshape(validation_features, (1000, 4*4*512))
test_features = np.reshape(test_features, (1000, 4*4*512))

In [5]:
test_dir 

'C:/Users/yang/Desktop/nongzuowu/small/test'

In [8]:
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, kernel_regularizer=regularizers.l2(0.001),
                                  activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 16,878,145
Trainable params: 16,878,145
Non-trainable params: 0
_________________________________________________________________


In [ ]:
conv_base.trainable=True


set_trainable=False
for layer in conv_base.layers:
    if layer.name=='block5_conv1':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False
        
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(
               rescale=1./255,
               rotation_range=40,
               width_shift_range=0.2,
               height_shift_range=0.2,
               shear_range=0.2,
               horizontal_flip=0.2,
               fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(150,150),
            batch_size=20,
            class_mode='binary')

validation_genetator = test_datagen.flow_from_directory(
            validation_dir,
            target_size=(150,150),
            batch_size = 20,
            class_mode = 'binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(train_generator, 
                              steps_per_epoch=127, 
                              epochs=100,
                              validation_data=validation_genetator,
                              validation_steps=50,
                              )

Found 2540 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
127/127 [==============================] - 62s 484ms/step - loss: 0.8032 - acc: 0.7366 - val_loss: 0.6850 - val_acc: 0.8150
Epoch 2/100
127/127 [==============================] - 61s 483ms/step - loss: 0.6574 - acc: 0.8154 - val_loss: 0.6197 - val_acc: 0.8520
Epoch 3/100
127/127 [==============================] - 62s 487ms/step - loss: 0.6099 - acc: 0.8441 - val_loss: 0.6335 - val_acc: 0.8270
Epoch 4/100
127/127 [==============================] - 62s 492ms/step - loss: 0.5622 - acc: 0.8650 - val_loss: 0.6061 - val_acc: 0.8510
Epoch 5/100
127/127 [==============================] - 62s 491ms/step - loss: 0.5390 - acc: 0.8740 - val_loss: 0.6069 - val_acc: 0.8480
Epoch 6/100
127/127 [==============================] - 63s 494ms/step - loss: 0.5084 - acc: 0.8835 - val_loss: 0.5412 - val_acc: 0.8800
Epoch 7/100
127/127 [===

127/127 [==============================] - 65s 509ms/step - loss: 0.1684 - acc: 0.9878 - val_loss: 0.5556 - val_acc: 0.9080
Epoch 59/100
127/127 [==============================] - 65s 508ms/step - loss: 0.1714 - acc: 0.9882 - val_loss: 0.6131 - val_acc: 0.9060
Epoch 60/100
127/127 [==============================] - 65s 510ms/step - loss: 0.1653 - acc: 0.9878 - val_loss: 0.7603 - val_acc: 0.8850
Epoch 61/100
127/127 [==============================] - 64s 508ms/step - loss: 0.1627 - acc: 0.9870 - val_loss: 0.5786 - val_acc: 0.9030
Epoch 62/100
127/127 [==============================] - 65s 511ms/step - loss: 0.1662 - acc: 0.9866 - val_loss: 0.5863 - val_acc: 0.9030
Epoch 63/100
127/127 [==============================] - 65s 508ms/step - loss: 0.1547 - acc: 0.9909 - val_loss: 0.5630 - val_acc: 0.9110
Epoch 64/100
127/127 [==============================] - 65s 511ms/step - loss: 0.1654 - acc: 0.9858 - val_loss: 0.5599 - val_acc: 0.9100
Epoch 65/100
127/127 [==============================] 

In [ ]:
model.save("hulianwang+(sample=2540&l2=0.001).h5")

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation accuracy')
plt.legend()


In [ ]:
test_generator=test_datagen.flow_from_directory(
                test_dir,
                target_size=(150, 150),
                batch_size=20,
                class_mode='binary')


test_loss, test_acc=model.evaluate_generator(test_generator, steps=50)

In [ ]:
test_acc